In [4]:
import os

os.chdir("../")  # chdir to repo root

In [9]:
import numpy as np
import pandas as pd

g_truth = pd.read_csv("data/gt-preprocessed/csv/test_big_size_A_B_E_K_WH_WB-points.csv")
g_truth

,images,x,y,labels
0,S_07_05_16_DSC00094.JPG,5862,626,3
1,S_07_05_16_DSC00162.JPG,1029,665,2
2,S_07_05_16_DSC00162.JPG,806,610,2
3,S_07_05_16_DSC00163.JPG,1305,2797,2
4,S_07_05_16_DSC00163.JPG,1082,2744,2
...,...,...,...,...
2294,42c89ea14cbcc5b6a90333988b6acac6f49371ed.JPG,4733,1619,6
2295,42c89ea14cbcc5b6a90333988b6acac6f49371ed.JPG,4784,1267,6
2296,4310a4722c6ff3128e2595b0dbfcaa95ea13d9c5.JPG,2145,1382,6
2297,4310a4722c6ff3128e2595b0dbfcaa95ea13d9c5.JPG,2269,643,6


In [28]:
g_truth_cnts = (
    g_truth.groupby(["images", "labels"])
    .agg({"x": "count"})
    .rename(columns={"x": "true_count"})
    .reset_index()
)
g_truth_cnts

,images,labels,true_count
0,01802f75da35434ab373569fffc1fd65a3417aef.JPG,6,9
1,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,6,11
2,02033bf9b6c41f5815072434f8d61707cc8ea1fb.JPG,6,1
3,02e306916552df0dfe01fa352590ebb5f2a8b8ab.JPG,6,4
4,045f3b931fae913307c1f11512e79ddd891cb3ad.JPG,6,3
...,...,...,...
276,S_07_05_16_DSC00694.JPG,3,1
277,S_07_05_16_DSC00752.JPG,3,1
278,S_07_05_16_DSC00753.JPG,3,1
279,S_07_05_16_DSC00810.JPG,2,12


In [30]:
predicted = pd.read_csv("data/test_results/herdnet_v2_full_imgs/detections.csv")
# predicted['labels'] = predicted['labels'].astype(int)
invalid_row_mask = predicted["labels"].isnull()
print(f"{sum(invalid_row_mask)} invalid rows found, out of {len(predicted)}, dropping them")
predicted = predicted.loc[~invalid_row_mask].copy()
predicted["labels"] = predicted["labels"].astype(int)

predicted_cnts = (
    predicted.groupby(["images", "labels"])
    .agg({"x": "count"})
    .rename(columns={"x": "pred_count"})
    .reset_index()
)
predicted_cnts

1 invalid rows found, out of 3905, dropping them


,images,labels,pred_count
0,01802f75da35434ab373569fffc1fd65a3417aef.JPG,1,1
1,01802f75da35434ab373569fffc1fd65a3417aef.JPG,2,1
2,01802f75da35434ab373569fffc1fd65a3417aef.JPG,6,10
3,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1,7
4,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,6,17
...,...,...,...
544,S_07_05_16_DSC00810.JPG,3,1
545,S_07_05_16_DSC00811.JPG,1,1
546,S_07_05_16_DSC00811.JPG,2,10
547,S_07_05_16_DSC00811.JPG,3,1


In [74]:
merged = g_truth_cnts.merge(predicted_cnts, on=["images", "labels"], how="outer")

merged["true_count"] = merged["true_count"].fillna(0.0)
merged["pred_count"] = merged["pred_count"].fillna(0.0)

# Plain counting error, can be positive or negative
merged["count_err"] = merged["pred_count"] - merged["true_count"]

# Absolute error : abs(count_error)
merged["abs_err"] = np.abs(merged["count_err"])

# Squared error
merged["sq_err"] = (merged["pred_count"] - merged["true_count"]) ** 2

# Count error as a percent of true count, only defined when true count > 0
merged["pct_err"] = np.where(
    merged["true_count"] > 0,
    100 * (merged["pred_count"] - merged["true_count"]) / merged["true_count"],
    np.nan,
)
merged["abs_pct_err"] = np.abs(merged["pct_err"])

merged.head(20)

,images,labels,true_count,pred_count,count_err,abs_err,sq_err,pct_err,abs_pct_err
0,01802f75da35434ab373569fffc1fd65a3417aef.JPG,1,0.0,1.0,1.0,1.0,1.0,NaN,NaN
1,01802f75da35434ab373569fffc1fd65a3417aef.JPG,2,0.0,1.0,1.0,1.0,1.0,NaN,NaN
2,01802f75da35434ab373569fffc1fd65a3417aef.JPG,6,9.0,10.0,1.0,1.0,1.0,11.111111,11.111111
3,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1,0.0,7.0,7.0,7.0,49.0,NaN,NaN
4,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,6,11.0,17.0,6.0,6.0,36.0,54.545455,54.545455
5,02033bf9b6c41f5815072434f8d61707cc8ea1fb.JPG,6,1.0,15.0,14.0,14.0,196.0,1400.000000,1400.000000
6,02e306916552df0dfe01fa352590ebb5f2a8b8ab.JPG,1,0.0,1.0,1.0,1.0,1.0,NaN,NaN
7,02e306916552df0dfe01fa352590ebb5f2a8b8ab.JPG,6,4.0,5.0,1.0,1.0,1.0,25.000000,25.000000
8,045f3b931fae913307c1f11512e79ddd891cb3ad.JPG,3,0.0,1.0,1.0,1.0,1.0,NaN,NaN
9,045f3b931fae913307c1f11512e79ddd891cb3ad.JPG,6,3.0,17.0,14.0,14.0,196.0,466.666667,466.666667


In [131]:
from animaloc_improved.tools.csv_tool import flatten_df_column_names

means_by_label = merged.groupby("labels").agg(
    {
        "true_count": ("mean", "sum"),
        "pred_count": ("mean", "sum"),
        "count_err": "mean",
        "abs_err": "mean",
        "sq_err": "mean",
        "pct_err": "mean",
        "abs_pct_err": "mean",
    }
)

means_by_label = flatten_df_column_names(means_by_label)
means_by_label

,true_count_mean,true_count_sum,pred_count_mean,pred_count_sum,count_err_mean,abs_err_mean,sq_err_mean,pct_err_mean,abs_pct_err_mean
labels,,,,,,,,,
1,4.891304,675.0,6.775362,935.0,1.884058,2.579710,12.826087,41.339635,53.456438
2,6.122807,349.0,6.140351,350.0,0.017544,2.052632,13.350877,7.061341,34.923635
3,3.481752,477.0,4.036496,553.0,0.554745,1.430657,5.459854,16.815252,36.866607
4,0.880952,74.0,2.130952,179.0,1.250000,2.059524,8.892857,41.413043,97.137681
5,2.400000,36.0,2.333333,35.0,-0.066667,1.533333,4.866667,-24.857955,24.857955
6,5.504000,688.0,14.816000,1852.0,9.312000,9.568000,398.704000,537.131553,538.563579


In [134]:
def _rename_agg_cols(aggregated: pd.DataFrame) -> pd.DataFrame:
    aggregated = flatten_df_column_names(aggregated)
    aggregated = aggregated.rename(
        columns={
            "true_count_mean": "M(C)",
            "true_count_sum": "Σ(C)",
            "pred_count_mean": "M(Ĉ)",
            "pred_count_sum": "Σ(Ĉ)",
            "count_err_mean": "ME",  # mean error, without taking abs value can be positive or neg
            "abs_err_mean": "MAE",  # mean absolute error
            "sq_err_mean": "MSE",  # mean squared error
            "pct_err_mean": "MPE",  # mean percent error
            "abs_pct_err_mean": "MAPE",
        }
    )
    for col in aggregated.columns:
        if col.startswith("M") or col.strip() == "RMSE":
            aggregated[col] = aggregated[col].round(1)

    return aggregated


means_by_label = _rename_agg_cols(means_by_label)
means_by_label["RMSE"] = means_by_label["MSE"] ** 0.5
means_by_label

,M(C),Σ(C),M(Ĉ),Σ(Ĉ),ME,MAE,MSE,MPE,MAPE,RMSE
labels,,,,,,,,,,
1,4.9,675.0,6.8,935.0,1.9,2.6,12.8,41.3,53.5,3.577709
2,6.1,349.0,6.1,350.0,0.0,2.1,13.4,7.1,34.9,3.660601
3,3.5,477.0,4.0,553.0,0.6,1.4,5.5,16.8,36.9,2.345208
4,0.9,74.0,2.1,179.0,1.2,2.1,8.9,41.4,97.1,2.983287
5,2.4,36.0,2.3,35.0,-0.1,1.5,4.9,-24.9,24.9,2.213594
6,5.5,688.0,14.8,1852.0,9.3,9.6,398.7,537.1,538.6,19.967474


In [133]:
!ls data/test_results/herdnet_v2_full_imgs/

after pyenv init
confusion_matrix.csv  metrics_results.csv
detections.csv        plots/


In [69]:
pd.read_csv("data/test_results/herdnet_v2_full_imgs/metrics_results.csv")

,class,species,n,recall,precision,f1_score,confusion,mae,mse,rmse,ap
0,1,Alcelaphinae,675,0.798519,0.576471,0.669565,0.069085,2.579710,12.826087,3.581353,0.666617
1,2,Buffalo,349,0.613181,0.611429,0.612303,0.147410,2.052632,13.350877,3.653885,0.498543
2,3,Kob,477,0.742138,0.640145,0.687379,0.221978,1.430657,5.459854,2.336633,0.645367
3,4,Warthog,74,0.432432,0.178771,0.252964,0.058824,2.059524,8.892857,2.982089,0.218422
4,5,Waterbuck,36,0.527778,0.542857,0.535211,0.269231,1.533333,4.866667,2.206052,0.470625
5,6,Elephant,688,0.654070,0.242981,0.354331,0.002217,9.568000,398.704000,19.967574,0.415106
6,binary,binary,2299,0.781209,0.460041,0.579075,0.000000,6.655039,209.189922,14.463399,0.516384


In [120]:
bin_metrics = (
    merged.groupby("images")
    .agg(
        {
            "true_count": "sum",
            "pred_count": "sum",
            "count_err": "sum",
        }
    )
    .reset_index()
)
# bin_metrics = flatten_df_column_names(bin_metrics)
bin_metrics["abs_err"] = np.abs(bin_metrics["count_err"])
bin_metrics["sq_err"] = bin_metrics["count_err"] ** 2
bin_metrics["pct_err"] = np.where(
    bin_metrics["true_count"] > 0,
    (bin_metrics["count_err"] / bin_metrics["true_count"]) * 100,
    np.nan,
)
bin_metrics["abs_pct_err"] = np.abs(bin_metrics["pct_err"])
bin_metrics

,images,true_count,pred_count,count_err,abs_err,sq_err,pct_err,abs_pct_err
0,01802f75da35434ab373569fffc1fd65a3417aef.JPG,9.0,12.0,3.0,3.0,9.0,33.333333,33.333333
1,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,11.0,24.0,13.0,13.0,169.0,118.181818,118.181818
2,02033bf9b6c41f5815072434f8d61707cc8ea1fb.JPG,1.0,15.0,14.0,14.0,196.0,1400.000000,1400.000000
3,02e306916552df0dfe01fa352590ebb5f2a8b8ab.JPG,4.0,6.0,2.0,2.0,4.0,50.000000,50.000000
4,045f3b931fae913307c1f11512e79ddd891cb3ad.JPG,3.0,18.0,15.0,15.0,225.0,500.000000,500.000000
...,...,...,...,...,...,...,...,...
253,S_07_05_16_DSC00694.JPG,1.0,1.0,0.0,0.0,0.0,0.000000,0.000000
254,S_07_05_16_DSC00752.JPG,1.0,2.0,1.0,1.0,1.0,100.000000,100.000000
255,S_07_05_16_DSC00753.JPG,1.0,2.0,1.0,1.0,1.0,100.000000,100.000000
256,S_07_05_16_DSC00810.JPG,12.0,15.0,3.0,3.0,9.0,25.000000,25.000000


In [122]:
bin_metrics

,images,true_count,pred_count,count_err,abs_err,sq_err,pct_err,abs_pct_err
0,01802f75da35434ab373569fffc1fd65a3417aef.JPG,9.0,12.0,3.0,3.0,9.0,33.333333,33.333333
1,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,11.0,24.0,13.0,13.0,169.0,118.181818,118.181818
2,02033bf9b6c41f5815072434f8d61707cc8ea1fb.JPG,1.0,15.0,14.0,14.0,196.0,1400.000000,1400.000000
3,02e306916552df0dfe01fa352590ebb5f2a8b8ab.JPG,4.0,6.0,2.0,2.0,4.0,50.000000,50.000000
4,045f3b931fae913307c1f11512e79ddd891cb3ad.JPG,3.0,18.0,15.0,15.0,225.0,500.000000,500.000000
...,...,...,...,...,...,...,...,...
253,S_07_05_16_DSC00694.JPG,1.0,1.0,0.0,0.0,0.0,0.000000,0.000000
254,S_07_05_16_DSC00752.JPG,1.0,2.0,1.0,1.0,1.0,100.000000,100.000000
255,S_07_05_16_DSC00753.JPG,1.0,2.0,1.0,1.0,1.0,100.000000,100.000000
256,S_07_05_16_DSC00810.JPG,12.0,15.0,3.0,3.0,9.0,25.000000,25.000000


In [127]:
bin_metrics["Especie"] = "binaria"
bin_metrics_agg = (
    bin_metrics.groupby("Especie")
    .agg(
        {
            "true_count": ("mean", "sum"),
            "pred_count": ("mean", "sum"),
            "abs_err": "mean",
            "sq_err": "mean",
            "pct_err": "mean",
            "abs_pct_err": "mean",
        }
    )
    .reset_index()
)

bin_metrics_agg

Especie true_count         pred_count           abs_err      sq_err  \
                 mean     sum       mean     sum      mean        mean   
0  binaria   8.910853  2299.0  15.131783  3904.0  6.655039  209.189922   

      pct_err abs_pct_err  
         mean        mean  
0  280.661843  283.693052

In [135]:
bin_metrics_agg = _rename_agg_cols(bin_metrics_agg)
bin_metrics_agg

,Especie,true_cnt_mean,true_cnt_total,M(Ĉ),pred_cnt_total,MAE,MSE,MPE,MAPE
0,binaria,8.910853,2299.0,15.1,3904.0,6.7,209.2,280.7,283.7


Toda la lógica de arriba empaquetada en un script:

In [11]:
!python animaloc_improved/tools/count_metrics_v2.py \
  --gt "data/gt-preprocessed/csv/test_big_size_A_B_E_K_WH_WB-points.csv" \
  --preds "data/test_results/herdnet_v2_full_imgs/detections.csv" \
  --out-dir "data/test_results/herdnet_v2_full_cnts_metrics_v2/"

after pyenv init
2025-11-17 15:42:13.860 | INFO     | __main__:_load_and_agg_g_truth:33 - Loaded ground truth data g_truth.shape=(2299, 4), unique images: g_truth['images'].nunique()=258
2025-11-17 15:42:13.865 | INFO     | __main__:_load_and_agg_preds:53 - Loaded prediction data predicted.shape=(3905, 13), unique images predicted['images'].nunique()=258
2025-11-17 15:42:13.865 | INFO     | __main__:_load_and_agg_preds:57 - 1 invalid rows found, out of 3905, dropping them
2025-11-17 15:42:13.869 | INFO     | __main__:main:210 - Writing merged data frame with gt-counts, pred-counts and errors by [Image, Especie] to: data/test_results/herdnet_v2_full_cnts_metrics_v2/g_truth_pred_cnts_errors_by_image_species.csv
2025-11-17 15:42:13.875 | INFO     | __main__:main:227 - Writing merged data frame with gt-counts, pred-counts and errors by [Image] to:data/test_results/herdnet_v2_full_cnts_metrics_v2/g_truth_pred_cnts_errors_by_image.csv
2025-11-17 15:42:13.886 | INFO     | __main__:main:236 - 

In [19]:
agg_metrics = pd.read_csv(
    "data/test_results/herdnet_v2_full_cnts_metrics_v2/aggregated_count_metrics_v2.csv"
)
agg_metrics = agg_metrics.drop("ME", axis=1)
agg_metrics = agg_metrics[["Σ(C)", "M(C)", "M(Ĉ)", "MAE", "RMSE", "MPE", "MAPE"]]
agg_metrics

,Σ(C),M(C),M(Ĉ),MAE,RMSE,MPE,MAPE
0,675.0,4.9,6.8,2.6,3.6,41.3,53.5
1,349.0,6.1,6.1,2.1,3.7,7.1,34.9
2,477.0,3.5,4.0,1.4,2.3,16.8,36.9
3,74.0,0.9,2.1,2.1,3.0,41.4,97.1
4,36.0,2.4,2.3,1.5,2.2,-24.9,24.9
5,688.0,5.5,14.8,9.6,20.0,537.1,538.6
6,2299.0,8.9,15.1,6.7,14.5,280.7,283.7


In [17]:
935 / 675, 6.8 / 4.9

(1.385185185185185, 1.387755102040816)